In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
data = pd.read_csv("record.csv")
columnsRate = ['rates']
columnsDuration = ['moveDurationNS', 'moveDurationWE',
       'yellowDurationNS', 'yellowDurationWE', 'turnDurationNS',
       'turnDurationWE', 'waitDurationNS', 'waitDurationWE']
columnsParams = ['leftOnlyNS', 'leftStraightNS', 'straightOnlyNS',
       'rightStraightNS', 'rightOnlyNS', 'allNS', 'leftOnlyWE',
       'leftStraightWE', 'straightOnlyWE', 'rightStraightWE', 'rightOnlyWE',
       'allWE', 'leftOutLanesNS', 'rightOutLanesNS', 'leftOutLanesWE',
       'rightOutLanesWE', 'lengthN',
       'lengthS', 'lengthW', 'lengthE', 'demandN', 'demandS', 'demandW',
       'demandE', 'demandProbNS_Straight', 'demandProbNS_Left',
       'demandProbNS_Right', 'demandProbNS_UTurn', 'demandProbWE_Straight',
       'demandProbWE_Left', 'demandProbWE_Right', 'demandProbWE_UTurn',
       'pDemandRegN', 'pDemandRegS', 'pDemandRegW', 'pDemandRegE',
       'pDemandOppN', 'pDemandOppS', 'pDemandOppW', 'pDemandOppE',
       'pSpeedRegN', 'pSpeedRegS', 'pSpeedRegW', 'pSpeedRegE', 'pSpeedOppN',
       'pSpeedOppS', 'pSpeedOppW', 'pSpeedOppE', 'outSpeedNS', 'outSpeedWE',
       'inSpeedNS', 'inSpeedWE', 'vehicleMaxSpeed', 'vehicleMinAccel',
       'vehicleMaxAccel', 'vehicleMinDecel', 'vehicleMaxDecel',
       'vehicleMinLength', 'vehicleMaxLength', 'minGap']
       
data_rate = torch.tensor(data[columnsRate].values, dtype=torch.float)
data_rate_mean = data_rate.mean()
data_rate_std = data_rate.std()
data_rate = (data_rate - data_rate.mean()) / data_rate.std()

data_duration = torch.tensor(data[columnsDuration].values, dtype=torch.float)
data_duration_min = data_duration.numpy().min(axis=0)
data_duration_max = data_duration.numpy().max(axis=0)
data_duration = (data_duration - data_duration.numpy().min(axis=0)) / (data_duration.numpy().max(axis=0) - data_duration.numpy().min(axis=0))

data_params = torch.tensor(data[columnsParams].values, dtype=torch.float)
data_params_min = data_params.numpy().min(axis=0)
data_params_max = data_params.numpy().max(axis=0)
data_params = (data_params - data_params.numpy().min(axis=0)) / (data_params.numpy().max(axis=0) - data_params.numpy().min(axis=0))

In [5]:
test = pd.read_csv("test.csv")
columnsRate = ['rates']
columnsDuration = ['moveDurationNS', 'moveDurationWE',
       'yellowDurationNS', 'yellowDurationWE', 'turnDurationNS',
       'turnDurationWE', 'waitDurationNS', 'waitDurationWE']
columnsParams = ['leftOnlyNS', 'leftStraightNS', 'straightOnlyNS',
       'rightStraightNS', 'rightOnlyNS', 'allNS', 'leftOnlyWE',
       'leftStraightWE', 'straightOnlyWE', 'rightStraightWE', 'rightOnlyWE',
       'allWE', 'leftOutLanesNS', 'rightOutLanesNS', 'leftOutLanesWE',
       'rightOutLanesWE', 'lengthN',
       'lengthS', 'lengthW', 'lengthE', 'demandN', 'demandS', 'demandW',
       'demandE', 'demandProbNS_Straight', 'demandProbNS_Left',
       'demandProbNS_Right', 'demandProbNS_UTurn', 'demandProbWE_Straight',
       'demandProbWE_Left', 'demandProbWE_Right', 'demandProbWE_UTurn',
       'pDemandRegN', 'pDemandRegS', 'pDemandRegW', 'pDemandRegE',
       'pDemandOppN', 'pDemandOppS', 'pDemandOppW', 'pDemandOppE',
       'pSpeedRegN', 'pSpeedRegS', 'pSpeedRegW', 'pSpeedRegE', 'pSpeedOppN',
       'pSpeedOppS', 'pSpeedOppW', 'pSpeedOppE', 'outSpeedNS', 'outSpeedWE',
       'inSpeedNS', 'inSpeedWE', 'vehicleMaxSpeed', 'vehicleMinAccel',
       'vehicleMaxAccel', 'vehicleMinDecel', 'vehicleMaxDecel',
       'vehicleMinLength', 'vehicleMaxLength', 'minGap']

test_rate = torch.tensor(test[columnsRate].values, dtype=torch.float)
test_rate = (test_rate - data_rate_mean) / data_rate_std

test_duration = torch.tensor(test[columnsDuration].values, dtype=torch.float)
test_duration = (test_duration - data_duration_min) / (data_duration_max - data_duration_min)

test_params = torch.tensor(test[columnsParams].values, dtype=torch.float)
test_params = (test_params - data_params_min) / (data_params_max - data_params_min)

NameError: ignored

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self, ninputA, ninputB, nhidden1, nhidden2):
    super().__init__()
    self.layer1A = nn.Linear(ninputA, nhidden1)
    self.layer1B = nn.Linear(ninputB, nhidden1)
    self.layer2 = nn.Linear(nhidden1 * 2, nhidden2)
    self.layer3 = nn.Linear(nhidden2, 1)
    self.loss_fn = nn.MSELoss() # Function to calculate the error of the network's predictions
    self.optimizer = optim.Adam(self.parameters()) # This function updates model parameters to minimize loss
    self.metrics = {'loss':[]} # Store statistics about how well the network is doing

  def forward(self, x):
    output1A = torch.sigmoid(self.layer1A(x[0]))
    output1B = torch.sigmoid(self.layer1B(x[1]))
    output1 = torch.cat([output1A, output1B], dim=1)
    output2 = torch.sigmoid(self.layer2(output1))
    return self.layer3(output2)
  
  def train(self, xA, xB, y, epochs, batch_size=64):
    for epoch in range(epochs):
      current_loss, n_steps = 0, 0
      batch_idxs = np.random.permutation(range(len(y))) # Good practice to shuffle the data every epoch
      for batch_start in range(0, len(y),batch_size):
        batch_idx = batch_idxs[batch_start:min(batch_start+batch_size,len(y))]
        batch_xA, batch_xB, batch_y = xA[batch_idx], xB[batch_idx], y[batch_idx]
        self.optimizer.zero_grad() # Clear out the old gradient information
        prediction = self(([batch_xA, batch_xB])) # Run the model and generate a prediction
        loss = self.loss_fn(prediction, batch_y)
        loss.backward() # Calculate the gradients
        current_loss += float(loss.detach().cpu().numpy())
        self.optimizer.step() # Use the gradients to update the model params
        n_steps += 1
      self.metrics['loss'].append(current_loss/n_steps)
    return self.metrics

In [ ]:
network = NeuralNetwork(len(columnsDuration), len(columnsParams), 100, 50)
network.train(data_duration, data_params, data_rate, 25)

{'loss': [0.7940798611286654,
  0.408867979025745,
  0.36695591629628677,
  0.3621205190817515,
  0.35855713043825693,
  0.358385796647474,
  0.35732248108310394,
  0.35646247768019096,
  0.3546803487711642,
  0.353527803856685,
  0.35346262779341164,
  0.35237603990667793,
  0.3535058056135254,
  0.3539236012591894,
  0.34891783664504206,
  0.3479791412272128,
  0.34965712777582036,
  0.3453074442334922,
  0.3504054557726565,
  0.34620261539417096,
  0.34443095320439243,
  0.34498024615178624,
  0.34596117259747533,
  0.3403295802782817,
  0.3392992328448468]}

In [ ]:
test_rate_predicted = network(([test_duration, test_params])).detach().numpy()[:,0]
px.scatter(x=test_rate.numpy()[:,0], y=test_rate_predicted)

In [ ]:
columnsParams = ['leftOnlyNS', 'leftStraightNS', 'straightOnlyNS',
       'rightStraightNS', 'rightOnlyNS', 'allNS', 'leftOnlyWE',
       'leftStraightWE', 'straightOnlyWE', 'rightStraightWE', 'rightOnlyWE',
       'allWE', 'leftOutLanesNS', 'rightOutLanesNS', 'leftOutLanesWE',
       'rightOutLanesWE', 'lengthN',
       'lengthS', 'lengthW', 'lengthE', 'demandN', 'demandS', 'demandW',
       'demandE', 'demandProbNS_Straight', 'demandProbNS_Left',
       'demandProbNS_Right', 'demandProbNS_UTurn', 'demandProbWE_Straight',
       'demandProbWE_Left', 'demandProbWE_Right', 'demandProbWE_UTurn',
       'pDemandRegN', 'pDemandRegS', 'pDemandRegW', 'pDemandRegE',
       'pDemandOppN', 'pDemandOppS', 'pDemandOppW', 'pDemandOppE',
       'pSpeedRegN', 'pSpeedRegS', 'pSpeedRegW', 'pSpeedRegE', 'pSpeedOppN',
       'pSpeedOppS', 'pSpeedOppW', 'pSpeedOppE', 'outSpeedNS', 'outSpeedWE',
       'inSpeedNS', 'inSpeedWE', 'vehicleMaxSpeed', 'vehicleMinAccel',
       'vehicleMaxAccel', 'vehicleMinDecel', 'vehicleMaxDecel',
       'vehicleMinLength', 'vehicleMaxLength', 'minGap']
       
px.histogram(x=data_params.detach().numpy()[:,3])

tensor([[-0.0257],
        [ 0.7479],
        [ 0.5259],
        ...,
        [ 0.3217],
        [-1.0758],
        [-0.6088]])


In [ ]:
fig = px.line(network.metrics,y='loss')
fig.show()

In [ ]:
class NeuralNetwork2(nn.Module):
  def __init__(self, ninputA, ninputB, nhidden1, nhidden2, oldnetwork):
    super().__init__()
    self.layer0 = nn.Linear(1, ninputA)
    self.layer1A = nn.Linear(ninputA, nhidden1)
    self.layer1B = nn.Linear(ninputB, nhidden1)
    self.layer2 = nn.Linear(nhidden1 * 2, nhidden2)
    self.layer3 = nn.Linear(nhidden2, 1)
    with torch.no_grad():
      self.layer1A.weight.copy_(oldnetwork.layer1A.weight)
      self.layer1A.require_grad = False
      self.layer1B.weight.copy_(oldnetwork.layer1B.weight)
      self.layer1B.require_grad = False
      self.layer2.weight.copy_(oldnetwork.layer2.weight)
      self.layer2.require_grad = False
      self.layer3.weight.copy_(oldnetwork.layer3.weight)
      self.layer3.require_grad = False
    self.optimizer = optim.Adam(self.parameters()) # This function updates model parameters to minimize loss
    self.metrics = {'loss':[]} # Store statistics about how well the network is doing

  def forward(self, x):
    duration = self.layer0(torch.ones((len(x[1]), 1)))
    output1A = torch.sigmoid(self.layer1A(duration))
    output1B = torch.sigmoid(self.layer1B(x[1]))
    output1 = torch.cat([output1A, output1B], dim=1)
    output2 = torch.sigmoid(self.layer2(output1))
    return self.layer3(output2)
  
  def train(self, xA, xB, epochs):
    for epoch in range(epochs):
      self.optimizer.zero_grad()
      prediction = -self(([xA, xB]))
      prediction.backward()
      self.optimizer.step()
    return self.metrics

In [ ]:
network2 = NeuralNetwork2(len(columnsDuration), len(columnsParams), 100, 50, network)

tensor([[ 0.4720],
        [-0.0944],
        [-0.7608],
        ...,
        [ 0.5055],
        [ 0.2049],
        [-0.0988]], grad_fn=<AddmmBackward0>)

In [ ]:
for i in range(len(test.index))
  network2.train(test_duration[i:i+1], test_params[i:i+1], 2000)
  network2.layer0.weight
  test_duration[0:1]